In [277]:
import json
import pandas as pd
import glob
from datetime import date, timedelta
import openpyxl
import pathlib
import numpy as np
from datetime import datetime
import os
import re
import datetime

In [278]:
#Regular / Appeal
from_DATE = '2023-11-05'
to_DATE = '2023-11-25'

In [279]:
from datetime import datetime
def convert_datetime(a):
    date_formats = ["%Y-%m-%d %H:%M:%S.%f", 
                    "%Y-%m-%d %H:%M:%S", 
                    "%m/%d/%Y %H:%M",
                    "%m-%d-%Y %H:%M", 
                    "%m/%d/%Y", 
                    "%Y-%m-%d", 
                    "%H:%M.%f",
                    "%m/%d/%Y %H:%M:%S"] 
    for fmt in date_formats:
        try:
            return datetime.strptime(str(a), fmt).strftime("%Y-%m-%d")
        except ValueError:
            pass
    try:
        return (datetime(1899, 12, 30) + timedelta(days=float(a))).strftime("%Y-%m-%d")
    except ValueError:
        pass
    if not re.match("\d+:\d+:\d+(\.\d+)?", str(a)):
        return str(a)
    return  
     
def input_data_excel( data_dir  ):
    data_dir = data_dir
    out_dir = '.'
    list_files = []
    for filename in pathlib.Path(data_dir).glob('**/*.xlsx'):
        list_files.append(filename)
    df1 = pd.concat(map(pd.read_excel, list_files),   ignore_index=True)
    return df1

def input_data_csv( data_dir  ):
    data_dir = data_dir
    out_dir = '.'
    list_files = []
    for filename in pathlib.Path(data_dir).glob('**/*.csv'):
        list_files.append(filename)
    df1 = pd.concat(map(pd.read_csv, list_files),   ignore_index=True)
    return df1

def input_data( data_dir  ):
    data_dir = data_dir
    out_dir = '.'
    list_files = []
    for filename in pathlib.Path(data_dir).glob('**/*.xlsx'):
        df=pd.concat(pd.read_excel(filename,sheet_name=None, skiprows=0))
        list_files.append(df)
    df1 = pd.concat(map(lambda file: pd.read_csv(file), list_files), ignore_index=True)
    return df1
def filter_time(df, column_name, from_date, to_date, selected_columns):
    filter_rawdata = df[(df[column_name].dt.strftime('%Y-%m-%d') >= from_date) & (df[column_name].dt.strftime('%Y-%m-%d') <= to_date)]
    filter_rawdata = filter_rawdata[selected_columns]
    return filter_rawdata
def merge_csv_files_upl(folder_path):
    file_paths = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.csv')]
    dfs = []

    for file_path in file_paths:
        try:
            df = pd.read_csv(file_path)
            df['source_file'] = os.path.basename(file_path)
            df.columns = df.columns.str.strip().str.lower().map(lambda x: re.sub(r'\s+', ' ', x))
            df.columns = df.columns.str.lower().str.replace(' ', '_')
            dfs.append(df)
        except Exception as e:
            print(f"Error reading file {file_path}: {str(e)}")
    merged_df = pd.concat(dfs, axis=0, ignore_index=True)
    cols_count = merged_df.columns.value_counts()
    duplicated_cols = cols_count[cols_count > 1].index.tolist()
    renamed_cols = {}
    for col in duplicated_cols:
        count = 1
        for idx in range(len(merged_df.columns)):
            if merged_df.columns[idx] == col:
                renamed_cols[col] = f"{col}.{count}"
                count += 1
    merged_df = merged_df.rename(columns=renamed_cols)
    return merged_df
def merge_csv_files_appeal(folder_path):
    file_paths = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.csv')]
    dfs = []
    for file_path in file_paths:
        try:
            df = pd.read_csv(file_path,dtype={'BPO_QA_Task ID': object,'Object_ID':object,'Queue _ID':object})
            df['BPO QA Date'] = df['BPO QA Date'].apply(convert_datetime)
            df['Week'] = os.path.basename(file_path).replace('.csv', '')
            dfs.append(df)
        except Exception as e:
            print(f"Error reading file {file_path}: {str(e)}")
    merged_df = pd.concat(dfs, axis=0, ignore_index=True)
    return merged_df
def merge_csv_files_sideproject(folder_path):
    file_paths = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.csv')]
    dfs = []
    for file_path in file_paths:
        try:
            df = pd.read_csv(file_path,dtype={'BPO_QA_Task ID': object,'Object_ID':object,'Queue _ID':object})
            df['Week'] = os.path.basename(file_path).replace('.csv', '')
            dfs.append(df)
        except Exception as e:
            print(f"Error reading file {file_path}: {str(e)}")
    merged_df = pd.concat(dfs, axis=0, ignore_index=True)
    return merged_df
def input_data( data_dir  ):
    data_dir = data_dir
    out_dir = '.'
    list_files = []
    for filename in pathlib.Path(data_dir).glob('**/*.xlsx'):
        df=pd.concat(pd.read_excel(filename,sheet_name=None, skiprows=0))
        list_files.append(df)
    df1 = pd.concat(list_files,   ignore_index=True)
    return df1

In [280]:
# Supporting 

#Queue
queue_list = pd.read_excel('Support_dashboard.xlsx',sheet_name='Queue List',dtype={'MOD QUEUE ID':object,'QA QUEUE ID':object})
queue_list = pd.DataFrame(queue_list)
queue_list = queue_list.dropna(subset=['QA QUEUE NAME'])

#Alternation
shadow_table = pd.read_excel('Support_dashboard.xlsx',sheet_name='Shadow')


#Week
week_list = pd.read_excel('Support_dashboard.xlsx',sheet_name='Week')
month_mapping = {
    1: 'Jan',
    2: 'Feb',
    3: 'Mar',
    4: 'Apr',
    5: 'May',
    6: 'Jun',
    7: 'Jul',
    8: 'Aug',
    9: 'Sep',
    10: 'Oct',
    11: 'Nov',
    12: 'Dec'
}
week_list['Month'] = week_list['Month'].map(month_mapping)

#Policy Errors
policy_errors_list = pd.read_excel('Support_dashboard.xlsx',sheet_name='Policy Errors').values
policy_errors_list = [item for sublist in policy_errors_list for item in sublist]

#PIP Tracker
pip_rawdata = pd.read_excel('Support_dashboard.xlsx',sheet_name='PIP_Tracker')

#Data full alternation
data_full_alternation = pd.read_excel('linemanager_full.xlsx')
data_full_alternation = data_full_alternation[data_full_alternation['Role'].str.contains('Operator')]
data_full_alternation = data_full_alternation[['EffectDate','Email','LineManager','Batch','Task','ProductionTenure','Role']]

In [281]:
#Check chuyển team IPR Alernation
data_full_alternation_copy = data_full_alternation[['EffectDate','Email','LineManager','Task']]
data_full_alternation_copy = data_full_alternation_copy.sort_values(by=['Email', 'EffectDate'], ascending=True)
current_date = datetime.now()
data_full_alternation_copy = data_full_alternation_copy[data_full_alternation_copy['EffectDate'] <= current_date]
data_full_alternation_copy['IPR_transfer'] = 'No'
mask_counterfeit = data_full_alternation_copy['Task'].isin(['Counterfeit_ds', 'Counterfeit_ns'])
counterfeit_rows = data_full_alternation_copy[mask_counterfeit]
mask_not_counterfeit = ~data_full_alternation_copy['Task'].isin(['Counterfeit_ds', 'Counterfeit_ns'])
not_counterfeit_rows = data_full_alternation_copy[mask_not_counterfeit]
last_task = None
for index, row in data_full_alternation_copy.iterrows():
    if last_task is None or row['Email'] != last_task['Email']:
        last_task = row
        continue
    if last_task['Task'] not in ['Counterfeit_ds', 'Counterfeit_ns'] and row['Task'] in ['Counterfeit_ds', 'Counterfeit_ns']:
        data_full_alternation_copy.at[index, 'IPR_transfer'] = 'Yes'
    last_task = row
ipr_transfer_tracker = data_full_alternation_copy.copy()
ipr_transfer_tracker.loc[ipr_transfer_tracker['IPR_transfer'] == 'Yes', 'IPR_Seniority'] = ipr_transfer_tracker.loc[ipr_transfer_tracker['IPR_transfer'] == 'Yes', 'EffectDate'] + timedelta(days=14)
ipr_transfer_tracker

,EffectDate,Email,LineManager,Task,IPR_transfer,IPR_Seniority
3797,2022-09-05,ai.btm@trans-cosmos.com.vn,NGUYỄN NGỌC TRÍ,0,No,NaT
3798,2022-09-06,ai.btm@trans-cosmos.com.vn,NGUYỄN NGỌC TRÍ,0,No,NaT
3799,2022-09-07,ai.btm@trans-cosmos.com.vn,NGUYỄN NGỌC TRÍ,0,No,NaT
3800,2022-09-08,ai.btm@trans-cosmos.com.vn,NGUYỄN NGỌC TRÍ,0,No,NaT
3801,2022-09-09,ai.btm@trans-cosmos.com.vn,NGUYỄN NGỌC TRÍ,0,No,NaT
...,...,...,...,...,...,...
209833,2023-05-18,yen.tp@trans-cosmos.com.vn,NGUYỄN ĐỖ PHÚ YÊN,Live_ds,No,NaT
209834,2023-05-19,yen.tp@trans-cosmos.com.vn,NGUYỄN ĐỖ PHÚ YÊN,Live_ds,No,NaT
209835,2023-05-20,yen.tp@trans-cosmos.com.vn,NGUYỄN ĐỖ PHÚ YÊN,Live_ds,No,NaT
209836,2023-05-21,yen.tp@trans-cosmos.com.vn,NGUYỄN ĐỖ PHÚ YÊN,Live_ds,No,NaT


In [282]:
shadow_table['SHADOW DATE'] = pd.to_datetime(shadow_table['SHADOW DATE'])
shadow_table['STARTING TIME'] = pd.to_datetime(shadow_table['STARTING TIME'], format='%H:%M:%S').dt.time
shadow_table['ENDING TIME'] = pd.to_datetime(shadow_table['ENDING TIME'], format='%H:%M:%S').dt.time
shadow_table['START DATE'] = pd.to_datetime(shadow_table['SHADOW DATE'].astype(str) + ' ' + shadow_table['STARTING TIME'].astype(str))
shadow_table['END DATE'] = pd.to_datetime(shadow_table['SHADOW DATE'].astype(str) + ' ' + shadow_table['ENDING TIME'].astype(str))
shadow_table_usefull = shadow_table[['PERFORMANCE LEVEL', 'NAME', 'MOD EMAIL','START DATE', 'END DATE']]
shadow_table_usefull

,PERFORMANCE LEVEL,NAME,MOD EMAIL,START DATE,END DATE
0,BOTTOM PERFORMANCE,PHẠM ĐOÀN ANH VI,vi.pda@trans-cosmos.com.vn,2023-09-19 14:15:00,2023-09-19 14:45:00
1,BOTTOM PERFORMANCE,TRẦN THIỆN TUẤN,tuan.tt@trans-cosmos.com.vn,2023-09-19 14:15:00,2023-09-19 14:35:00
2,TOP PERFORMANCE,NGUYỄN CHÍ TÀI,tai.nc@trans-cosmos.com.vn,2023-09-19 00:30:00,2023-09-19 00:50:00
3,TOP PERFORMANCE,NGUYỄN NGÔ TRƯỜNG DUY,duy.nnt@trans-cosmos.com.vn,2023-09-19 14:25:00,2023-09-19 14:45:00
4,TOP PERFORMANCE,LÂM TRÚC MAI,mai.lt1@trans-cosmos.com.vn,2023-09-20 15:35:00,2023-09-20 16:00:00
5,BOTTOM PERFORMANCE,ĐỖ THỊ THÙY TRANG,trang.dtt1@trans-cosmos.com.vn,2023-09-21 22:30:00,2023-09-21 22:45:00
6,TOP PERFORMANCE,ĐOÀN THỊ XUÂN HƯƠNG,huong.dtx@trans-cosmos.com.vn,2023-09-21 14:50:00,2023-09-21 15:20:00
7,BOTTOM PERFORMANCE,TRƯƠNG NGỌC QUANG,quang.tn@trans-cosmos.com.vn,2023-09-22 16:50:00,2023-09-22 17:20:00
8,BOTTOM PERFORMANCE,VÕ THỊ BÍCH PHƯƠNG,phuong.vtb3@trans-cosmos.com.vn,2023-09-22 17:25:00,2023-09-22 17:55:00
9,TOP PERFORMANCE,ƯNG THÁI THÀNH,thanh.ut@trans-cosmos.com.vn,2023-09-22 17:30:00,2023-09-22 18:00:00


In [283]:
# shadow_rawdata = input_data_csv('D:/New folder (3)')
# shadow_rawdata = shadow_rawdata[['title', 'task_id', '1_verifier', '1_resolve_time','1_duration']]
# merged_df = shadow_table_usefull.merge(shadow_rawdata, left_on='MOD EMAIL', right_on='1_verifier')
# shadow_df = merged_df[(merged_df['1_resolve_time'] >= merged_df['START DATE']) & (merged_df['1_resolve_time'] <= merged_df['END DATE'])]
# shadow_df = shadow_df.groupby(by=['MOD EMAIL','title','START DATE','END DATE'],as_index=False).agg({'task_id':'count','1_duration':'mean'})
# shadow_df.to_excel('shadow.xlsx')

In [284]:
# pip_rawdata_clean = pd.melt(pip_rawdata, id_vars=['Mod Email', 'Superior', 'Task', 'PIP Level Progress', 'PIP Stage Progress', 'Team', 'QA'],
#                       value_vars=['Stage 1', 'Stage 2', 'Stage 3', 'Stage 4', 'Stage 5', 'Stage 6', 'Stage 7'],
#                       var_name='Stage', value_name='PIP (Week) Processing')
# pip_rawdata_clean = pip_rawdata_clean.dropna(subset=['PIP (Week) Processing'])
# pip_rawdata_clean = pd.merge(pip_rawdata_clean,week_list,how='left',left_on='PIP (Week) Processing',right_on='Mod_Week')
# pip_rawdata_clean[['PIP (Week) Processing','Date','Mod Email', 'Superior', 'Task', 'PIP Level Progress','PIP Stage Progress', 'Team', 'QA']]

In [285]:
team = {
    "Regular":['Livestream', 'IPR', 'Seller', 'Anchor Video','CB Product','LL Product'],
    "Appeal":['Appeal'],
    "Side Project":['Side Project'],
    "E-UPL":['E-UPL'],
    "G-UPL":['G-UPL'],
}

aliases_final_decision = {"Failed appeal":['failed appeal','mod wrong','mods wrong','r1 wrong','r2 wrong'] ,
            "Edge case":['edge case', 'borderline','boderline'],
            "Appeal successfully":['appeal successfully','qa wrong','qas wrong','mod correct','bo wrong','tl wrong'],
            }

aliases = {
    "Not Approve": ['"audit_status":400','"status":400','":400,"','400','disapproval', 'disapproved','not approval','not approve' ,'notapproval','disapprove','not approved','violation','reject'],
    "Approve": ['"audit_status":200','approve','"status":200' ,'approval', 'approved','200','general']
}

filter_week_func = {'W44','W45','W46','W47'}
#'W1','W2','W3','W4','W5','W6','W7','W8','W9','W10','W11','W12','W13','W14','W15','W16','W17','W18','W19','W20','W21','W22','W23','W24','W25','W26','W27','W28','W29','W30','W31','W32','W33','W34','W35','W36','W37','W38','W39','W40'
def filter_week(df,W,cp):
    df = df[df['Week'].str.contains('|'.join(W))]
    df = df[df['COMPOUND'].isin(cp)]
    return df

**LIVE PRODUCTIVITY**

In [286]:
# live_prod_rawdata = input_data_csv('Live - Prod by date')
# live_prod_rawdata['Date'] = pd.to_datetime(live_prod_rawdata['Date'].apply(convert_datetime))
# live_prod_rawdata = pd.merge(live_prod_rawdata,queue_list,how='left',left_on='Queue Title',right_on='MOD QUEUE NAME')
# live_prod = pd.merge(live_prod_rawdata,week_list,how='left',on='Date')
# live_prod = live_prod[['Mod_Week','Date', 'Queue Title','QA QUEUE NAME', 'Moderator', 'Queue ID','No. of Moderation Tasks']]
# live_prod

In [287]:
People_statistic_append = input_data('People Daily')
People_statistic_append= People_statistic_append[People_statistic_append['People']!="汇总/All"]
People_Round_rawdata = People_statistic_append[['Selected duration',
                                      'Sampling Queue',
                                      'People',
                                      'No. of Samples']]
People_Round_rawdata.rename(columns={'People':'Moderators Name',
                             'Sampling Queue':'Queue Name',
                             'No. of Samples':'Sample Size'},inplace=True)
#Split duration
People_Round_rawdata[['Moderation Date','End date']] = People_Round_rawdata['Selected duration'].str.split('~', expand=True, n=1)
People_Round_rawdata['Moderation Date'] = pd.to_datetime(People_Round_rawdata['Moderation Date'].apply(convert_datetime))
People_Round_rawdata = People_Round_rawdata.drop_duplicates()
People_Round_rawdata = pd.merge(People_Round_rawdata,week_list,how='left',left_on='Moderation Date',right_on='Date')
People_Round_rawdata = People_Round_rawdata[['Mod_Week','Moderation Date','Queue Name','Moderators Name','Sample Size']].rename(columns={'Mod_Week':'Week'})
People_Round = People_Round_rawdata.groupby(by=['Week','Moderation Date','Queue Name','Moderators Name'],as_index=False).agg({'Sample Size':'sum'})
for week in People_Round['Week'].unique():
    df = People_Round[People_Round['Week'] == week]
    df.to_csv(f'Sample Size All Queues/Regular/{week}.csv', index=False)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_7908\2114196068.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  People_Round_rawdata.rename(columns={'People':'Moderators Name',
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_7908\2114196068.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  People_Round_rawdata[['Moderation Date','End date']] = People_Round_rawdata['Selected duration'].str.split('~', expand=True, n=1)
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_7908\2114196068.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

In [288]:
#Regular_appeal_rawdata_diff_case

regular_appeal_Rawdata = merge_csv_files_appeal('Regular_appeal')
regular_appeal_Rawdata = regular_appeal_Rawdata[~regular_appeal_Rawdata['Queue Name'].isnull()]
regular_appeal_Rawdata['mod_date'] = pd.to_datetime(regular_appeal_Rawdata['mod_date'].apply(convert_datetime))
regular_appeal_Rawdata['Final Decision'] = regular_appeal_Rawdata['Final Decision'].fillna('Mod correct')
for k, v in aliases_final_decision.items():
    pat = '|'.join(v)
    regular_appeal_Rawdata.loc[regular_appeal_Rawdata['Final Decision'].str.contains(pat, case=False),'Final Decision'] = k
BPO_QA_Decision_keep_values = ['Failed appeal', 'Appeal successfully', 'Edge case','Mod correct']
temp_col = regular_appeal_Rawdata['Final Decision'].copy()
temp_col[~temp_col.isin(BPO_QA_Decision_keep_values)] = 'Failed appeal'
regular_appeal_Rawdata['Final Decision'] = temp_col
regular_appeal_diff_cases = regular_appeal_Rawdata[(regular_appeal_Rawdata['Final Decision']=="Failed appeal") | (regular_appeal_Rawdata['Final Decision']=="Edge case")]
regular_appeal_diff_cases['mod_date'] = pd.to_datetime(regular_appeal_diff_cases['mod_date'].apply(convert_datetime))
regular_appeal_diff_cases['Week'].value_counts()

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_7908\913736312.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  regular_appeal_diff_cases['mod_date'] = pd.to_datetime(regular_appeal_diff_cases['mod_date'].apply(convert_datetime))


Week
W1 (12.31 - 01.06)     1082
W13 (03.25 - 03.31)    1030
W2 (01.07 - 01.13)      832
W8 (02.18 - 02.24)      719
W5 (01.28 - 02.03)      692
W3 (01.14 - 01.20)      663
W9 (02.25 - 03.03)      656
W7 (02.11 - 02.17)      642
W10 (03.04 - 03.10)     616
W6 (02.04 - 02.10)      603
W21 (05.22 - 05.28)     562
W11 (03.11 - 03.17)     546
W4 (01.21 - 01.27)      543
W16 (04.17 - 04.23)     542
W17 (04.24 - 04.30)     541
W12 (03.18 - 03.24)     539
W20 (05.15 - 05.21)     514
W25 (06.19 - 06.25)     509
W23 (06.05 - 06.11)     498
W22 (05.29 - 05.06)     494
W16 (04.15 - 04.21)     485
W14 (04.01 - 04.07)     474
W15 (04.08 - 04.14)     462
W24 (06.12 - 06.18)     450
W18 (05.01 - 05.07)     435
W26 (06.26 - 07.02)     428
W19 (05.08 - 05.14)     353
W28 (07.09 - 07.15)     287
W27 (07.03 - 07.08)     264
W29 (07.16 - 07.22)     259
W32 (08.06 - 08.12)     255
W38 (09.17 - 09.23)     252
W31 (07.30 - 08.05)     248
W34 (08.20 - 08.26)     243
W35 (08.27 - 09.02)     239
W46 (11.12 - 11

In [289]:
#Regular
regular_sample_size = input_data_csv('Sample Size All Queues/Regular')
regular_sample_size ['Moderation Date'] = pd.to_datetime(regular_sample_size ['Moderation Date'].apply(convert_datetime))
regular_sample_size = regular_sample_size.drop_duplicates()

In [290]:
#Diff case - Regular
regular_mod_wrong = regular_appeal_diff_cases.dropna(subset=['Final Decision'])
regular_mod_wrong = regular_appeal_diff_cases.loc[(~regular_appeal_diff_cases['Queue Name'].str.contains(r"\b.ppeal .udit\b")) & (regular_appeal_diff_cases['Final Decision'] == "Failed appeal")]
regular_mod_wrong['mod_date'] = pd.to_datetime(regular_mod_wrong['mod_date'].apply(convert_datetime))
regular_mod_wrong = regular_mod_wrong.groupby(by=['mod_date','Queue Name','mod'], as_index=False).agg({'object id':'count'})
regular_mod_wrong = regular_mod_wrong[['mod_date','Queue Name','mod','object id']].rename(columns={'mod_date':'Moderation Date',
                                                                                                      'Queue Name':'Queue Name',
                                                                                                      'mod':'Moderators Name',
                                                                                                      'object id':'Mods Wrong'})
regular_mod_wrong['Moderation Date'] = pd.to_datetime(regular_mod_wrong['Moderation Date'])
regular_mod_wrong = pd.merge(regular_mod_wrong,week_list,how='left',left_on='Moderation Date',right_on='Date').rename(columns={'Mod_Week':'Week'})
regular_mod_wrong = regular_mod_wrong[['Week','Moderation Date','Queue Name','Moderators Name','Mods Wrong']]                
for week in regular_mod_wrong['Week'].unique():
    df = regular_mod_wrong[regular_mod_wrong['Week'] == week]
    df.to_csv(f'False case All Queues/Regular/{week}.csv', index=False)       

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_7908\3341213000.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  regular_mod_wrong['mod_date'] = pd.to_datetime(regular_mod_wrong['mod_date'].apply(convert_datetime))


In [291]:
#Appeal
qa_raw_appeal = input_data_csv('Appeal daily')
qa_raw_appeal['1_verifier'] = qa_raw_appeal['1_verifier'].str.replace('robot_estimate@','')
qa_raw_appeal = qa_raw_appeal[qa_raw_appeal['1_verifier'].str.contains('@trans-cosmos.com.vn')]
qa_raw_appeal = qa_raw_appeal[qa_raw_appeal['2_resolve_time'].notnull()]
qa_raw_appeal = qa_raw_appeal.drop_duplicates(subset='task_id',keep='last')
qa_raw_appeal['1_resolve_time'] = pd.to_datetime(qa_raw_appeal['1_resolve_time'].apply(convert_datetime))
qa_raw_appeal['2_resolve_time'] = pd.to_datetime(qa_raw_appeal['2_resolve_time'].apply(convert_datetime))
appeal_sample_size_ex = qa_raw_appeal.groupby(by=['1_resolve_time','title','1_verifier'],as_index=False).agg({'task_id':'count'}).sort_values(by=['1_resolve_time'],ascending=True).rename(columns={'1_resolve_time':'Moderation Date','title':'Queue Name','1_verifier':'Moderators Name','task_id':'Sample Size'})
appeal_sample_size_ex = pd.merge(appeal_sample_size_ex,week_list,how='left',left_on='Moderation Date',right_on='Date')
appeal_sample_size_ex = appeal_sample_size_ex[['Mod_Week','Moderation Date','Queue Name','Moderators Name','Sample Size']].rename(columns={'Mod_Week':'Week'})
for week in appeal_sample_size_ex['Week'].unique():
    df = appeal_sample_size_ex[appeal_sample_size_ex['Week'] == week]
    df.to_csv(f'Sample Size All Queues/Appeal/{week}.csv', index=False)

In [292]:
appeal_sample_size = input_data_csv('Sample Size All Queues/Appeal')
appeal_sample_size ['Moderation Date'] = pd.to_datetime(appeal_sample_size ['Moderation Date'].apply(convert_datetime))
appeal_sample_size = appeal_sample_size.drop_duplicates()

In [293]:
#Diff case - Appeal
appeal_mod_wrong = regular_appeal_diff_cases.loc[(regular_appeal_diff_cases['Queue Name'].str.contains(r"\b.ppeal .udit\b")) & (regular_appeal_diff_cases['Final Decision']=="Failed appeal")]
appeal_mod_wrong['mod_date'] = appeal_mod_wrong['mod_date'].apply(convert_datetime)
appeal_mod_wrong = appeal_mod_wrong.groupby(by=['mod_date','Queue Name','mod'],as_index=False).agg({'object id':'count'})
appeal_mod_wrong = appeal_mod_wrong[['mod_date','Queue Name','mod','object id']].rename(columns={'mod_date':'Moderation Date',
                                                                                                      'Queue Name':'Queue Name',
                                                                                                      'mod':'Moderators Name',
                                                                                                      'object id':'Mods Wrong'})
appeal_mod_wrong['Moderation Date'] = pd.to_datetime(appeal_mod_wrong['Moderation Date'])
appeal_mod_wrong = pd.merge(appeal_mod_wrong,week_list,how='left',left_on='Moderation Date',right_on='Date').rename(columns={'Mod_Week':'Week'})
appeal_mod_wrong = appeal_mod_wrong[['Week','Moderation Date','Queue Name','Moderators Name','Mods Wrong']]
for week in appeal_mod_wrong['Week'].unique():
    df = appeal_mod_wrong[appeal_mod_wrong['Week'] == week]
    df.to_csv(f'False case All Queues/Appeal/{week}.csv', index=False)       

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_7908\2329938305.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  appeal_mod_wrong['mod_date'] = appeal_mod_wrong['mod_date'].apply(convert_datetime)


**NEWBIE INFLUENCE**

In [294]:
failed_edge_cases = regular_appeal_Rawdata.copy()
print(failed_edge_cases.columns)
failed_edge_cases['BPO QA Date'] = pd.to_datetime(failed_edge_cases['BPO QA Date'].apply(convert_datetime))
failed_edge_cases = failed_edge_cases.groupby(['Reported Week','Compound','BPO QA Date','Queue Name','mod'], as_index=False).apply(lambda x: pd.Series({
    'Edge case': (x['Final Decision'] == 'Edge case').sum(),
    'Failed appeal': (x['Final Decision'] == 'Failed appeal').sum()
}))
mod_wrong_temp = pd.merge(failed_edge_cases,queue_list,how='left',left_on='Queue Name',right_on='QA QUEUE NAME')
mod_wrong_temp = pd.merge(mod_wrong_temp,data_full_alternation,how='left',left_on=['BPO QA Date','mod'],right_on=['EffectDate','Email'])
mod_wrong_temp = mod_wrong_temp[mod_wrong_temp['Reported Week'].str.contains('|'.join(filter_week_func))]
mod_wrong_temp = mod_wrong_temp[['Reported Week','Compound','Queue Name','mod','Batch','ProductionTenure','Edge case','Failed appeal']]
mod_wrong_temp.to_excel('newbie influence tracker.xlsx',index=False)
pv_mod_wrong_temp = pd.pivot_table(mod_wrong_temp,values=['Edge case','Failed appeal'],index=['Compound'],columns='Reported Week',aggfunc='sum')
pv_mod_wrong_temp = pv_mod_wrong_temp.fillna("---")
pv_mod_wrong_temp = pv_mod_wrong_temp.reset_index()
pv_mod_wrong_temp

Index(['Reported Week', 'Compound', 'Queue Name', 'Queue ID', 'mod', 'TL',
       'mod_date', 'object id', 'mod_result', 'mod_reason', 'mod_aht',
       'BPO QA', 'BPO QA Date', 'BPO QA Task ID', 'BPO QA Link',
       'BPO QA Result', 'BPO QA Reason', 'Diff case', 'To be appeal (Y/N)',
       '[TL Remarks]', 'BPO QA Handler', 'Final Decision', 'Final False Error',
       'Policy Error', 'Error Category 1', 'Error Category 2',
       'Error Category 3', 'Mod RCA from Rock', 'Remark', 'Screenshot',
       '[BD] QA Handler', 'BD QA Agree/Disagree', 'BD QA Approve/ Reject',
       'BD QA Decision', 'Final False Error.1', 'Wrong Tagging',
       '[BD] Comments', 'Screenshot.1', 'Policy screenshot', 'Week', 'Month',
       'Date & Time Dump', '[BD] Remarks', 'Policy Screenshot', 'Date',
       'RCA lv1', 'RCA lv2', 'RCA lv3', 'Policy Error.1', 'Policy error',
       'Policy error_1', 'Policy error_2', 'Final_False_Error',
       'Wrong_Tagging', 'LineManager', '#REF!', '#REF!.1', '#REF!.2',


Compound           Edge case                      \
Reported Week               W44 (10.29 - 11.04) W45 (11.05 - 11.11)   
0              Anchor Video                   5                   3   
1                    Appeal                   2                   0   
2                CB Product                   1                   1   
3                       IPR                   6                   5   
4                LL Product                   1                   1   
5                Livestream                   4                   6   
6                    Seller                   5                   3   

                                                            Failed appeal  \
Reported Week W46 (11.12 - 11.18) W47 (11.19 - 11.25) W44 (10.29 - 11.04)   
0                               3                   2                  32   
1                               0                   0                   8   
2                               2                   0                  13   
3                               8                   4                  42   
4                               6                   3                  51   
5                               4                   4                  18   
6                               8                   5                  10   

                                                                           
Reported Week W45 (11.05 - 11.11) W46 (11.12 - 11.18) W47 (11.19 - 11.25)  
0                              40                  37                  33  
1                               6                   6                  18  
2                              11                  16                  10  
3                              41                  48                  58  
4                              45                  61                  67  
5                              19                  27                   9  
6                               8                  12                   5

In [295]:
#Shop UPL
E_UPL_rawdata = merge_csv_files_upl('E_UPL')
E_UPL_rawdata['Queue Name'] = E_UPL_rawdata['queue_name']
E_UPL_rawdata['Moderators Name'] = E_UPL_rawdata['mod_name'].fillna(E_UPL_rawdata['1st_mod_name'])
E_UPL_rawdata['Moderators Name 2'] = E_UPL_rawdata['2nd_mod_name']
E_UPL_rawdata['Count'] = E_UPL_rawdata['object_id']
E_UPL_rawdata['Moderation Date'] = pd.to_datetime(E_UPL_rawdata['1st_date'].apply(convert_datetime))
E_UPL_rawdata['QA Date'] = pd.to_datetime(E_UPL_rawdata['2nd_date'].apply(convert_datetime))
E_UPL_rawdata['Error Category 1'] = E_UPL_rawdata['error_category_1']
E_UPL_rawdata['Error Category 2'] = E_UPL_rawdata['error_category_2']
E_UPL_rawdata['Error Category 3'] = E_UPL_rawdata['error_category_3']

E_UPL_rawdata= E_UPL_rawdata[~E_UPL_rawdata['Queue Name'].isnull()]
E_UPL_rawdata = E_UPL_rawdata.sort_values(by='Moderation Date', ascending=True)
def adjust_results(row):
    if pd.isnull(row['3rd_result']):
        row['QA Result'] = row['2nd_result']
        row['QA Reason'] = row['2nd_rejection_reason']
        row['Mod 2 Result'] = None
        row['Mod 2 Reason'] = None
        row['Mod 1 Result'] = row['1st_result']
        row['Mod 1 Reason'] = row['1st_rejection_reason']
    else:
        row['QA Result'] = row['3rd_result']
        row['QA Reason'] = row['3rd_rejection_reason']
        row['Mod 2 Result'] = row['2nd_result']
        row['Mod 2 Reason'] = row['2nd_rejection_reason']
        row['Mod 1 Result'] = row['1st_result']
        row['Mod 1 Reason'] = row['1st_rejection_reason']
    return row

E_UPL_rawdata = E_UPL_rawdata.apply(adjust_results, axis=1)
E_UPL_rawdata = pd.merge(E_UPL_rawdata,week_list,how='left',left_on='Moderation Date',right_on='Date').rename(columns={'UPL_Week':'Week'})
def fill_final_decision(row):
    valid_values = ['Failed appeal', 'Edge case', 'Appeal successfully']
    for column in ['final_decision_(pbo)', 'bd_qa_decision', '[bpo_qa]_decision']:
        if row[column] in valid_values:
            return row[column]
        elif pd.notnull(row[column]):
            return row[column]
    return ''
E_UPL_rawdata['Final Decision'] = E_UPL_rawdata.apply(fill_final_decision, axis=1)
E_UPL_rawdata['Final Decision'] = E_UPL_rawdata['Final Decision'].fillna('Mod correct')
for k, v in aliases_final_decision.items():
    pat = '|'.join(v)
    E_UPL_rawdata.loc[E_UPL_rawdata['Final Decision'].str.contains(pat, case=False),'Final Decision'] = k
def set_policy_error(row):
    if row['Final Decision'] in ['Failed appeal', 'Edge case']:
        return row['2nd_rejection_reason']
    else:
        return row['1st_rejection_reason']
E_UPL_rawdata['Policy Error'] = E_UPL_rawdata.apply(set_policy_error, axis=1) 
def check_approval_status(value):
    for status, keywords in aliases.items():
        found_keywords = [keyword for keyword in keywords if keyword in value.lower()]
        if found_keywords:
            return status
    return None
columns_to_check = ['Mod 1 Result', 'Mod 2 Result', 'QA Result']
for column in columns_to_check:
    E_UPL_rawdata[column] = E_UPL_rawdata[column].apply(lambda x: check_approval_status(x) if isinstance(x, str) else np.nan)            
def get_moderation_error_round(row):
    if row['Final Decision'] == 'Failed appeal':
        if row['Mod 1 Result'] != row['QA Result']:
            return '1st wrong'
        elif row['Mod 2 Result'] != row['QA Result']:
            return '2nd wrong'
    return ''
E_UPL_rawdata['Moderation_Error_Round'] = E_UPL_rawdata.apply(get_moderation_error_round, axis=1)

E_UPL_lineup = E_UPL_rawdata[['Moderation Date','Queue Name','Moderators Name','Moderators Name 2','Count','Moderation_Error_Round']]
E_UPL_lineup_1 = E_UPL_lineup[['Moderation Date','Queue Name','Moderators Name','Count','Moderation_Error_Round']]
E_UPL_lineup_2 = E_UPL_lineup[['Moderation Date','Queue Name','Moderators Name 2','Count','Moderation_Error_Round']]

E_UPL_lineup_1_sample_size = E_UPL_lineup_1.groupby(['Moderation Date', 'Queue Name', 'Moderators Name'], as_index=False)['Count'].count()
E_UPL_lineup_2_sample_size = E_UPL_lineup_2.groupby(['Moderation Date', 'Queue Name', 'Moderators Name 2'], as_index=False)['Count'].count()
E_UPL_lineup_sample_size = pd.concat([E_UPL_lineup_1_sample_size, E_UPL_lineup_2_sample_size], ignore_index=True)
E_UPL_lineup_sample_size['Moderators Name'] = E_UPL_lineup_sample_size['Moderators Name'].fillna(E_UPL_lineup_sample_size['Moderators Name 2'])
E_UPL_lineup_sample_size.drop(['Moderators Name 2'], axis=1, inplace=True)

E_UPL_sample_size = pd.merge(E_UPL_lineup_sample_size,week_list,how='left',left_on='Moderation Date',right_on='Date')
E_UPL_sample_size = E_UPL_sample_size.rename(columns={'Count': 'Sample Size','UPL_Week':'Week'})
E_UPL_sample_size = E_UPL_sample_size[['Week','Moderation Date','Queue Name','Moderators Name','Sample Size']]
for week in E_UPL_sample_size['Week'].unique():
    df = E_UPL_sample_size[E_UPL_sample_size['Week'] == week]
    df.to_csv(f'Sample Size All Queues/E_UPL/{week}.csv', index=False)
E_UPL_lineup_1_mod_wrong = E_UPL_lineup_1.groupby(['Moderation Date', 'Queue Name', 'Moderators Name'])['Moderation_Error_Round'].apply(lambda x: ((x == '1st wrong')).sum()).reset_index(name='Mods Wrong')
E_UPL_lineup_2_mod_wrong = E_UPL_lineup_2[~E_UPL_lineup_2['Moderators Name 2'].isnull()]
E_UPL_lineup_2_mod_wrong = E_UPL_lineup_2_mod_wrong.groupby(['Moderation Date', 'Queue Name', 'Moderators Name 2'])['Moderation_Error_Round'].apply(lambda x: ((x == '2nd wrong')).sum()).reset_index(name='Mods Wrong')
E_UPL_lineup_mod_wrong = pd.concat([E_UPL_lineup_1_mod_wrong, E_UPL_lineup_2_mod_wrong], ignore_index=True)
E_UPL_lineup_mod_wrong['Moderators Name'] = E_UPL_lineup_mod_wrong['Moderators Name'].fillna(E_UPL_lineup_mod_wrong['Moderators Name 2'])
E_UPL_lineup_mod_wrong.drop(['Moderators Name 2'], axis=1, inplace=True)
E_UPL_mod_wrong = pd.merge(E_UPL_lineup_mod_wrong,week_list,how='left',left_on='Moderation Date',right_on='Date')
E_UPL_mod_wrong = E_UPL_mod_wrong.rename(columns={'UPL_Week':'Week'})
E_UPL_mod_wrong = E_UPL_mod_wrong[['Week','Moderation Date','Queue Name','Moderators Name','Mods Wrong']]
for week in E_UPL_mod_wrong['Week'].unique():
    df = E_UPL_mod_wrong[E_UPL_mod_wrong['Week'] == week]
    df.to_csv(f'False case All Queues/E_UPL/{week}.csv', index=False)

In [296]:
#Gen UPL
G_UPL_rawdata = merge_csv_files_upl('G_UPL')
G_UPL_rawdata['Queue Name'] =  G_UPL_rawdata['queue_name'].fillna(G_UPL_rawdata['title'])
G_UPL_rawdata['Moderators Name'] = G_UPL_rawdata['1st_name'].fillna(G_UPL_rawdata['round_1'])
G_UPL_rawdata['Moderators Name 2'] = G_UPL_rawdata['round_2']
G_UPL_rawdata['Count'] = G_UPL_rawdata['object_id']
G_UPL_rawdata['Moderation Date'] = pd.to_datetime(G_UPL_rawdata['1st_date'].apply(convert_datetime)).fillna(pd.to_datetime(G_UPL_rawdata['1_resolve_time'].apply(convert_datetime)))
G_UPL_rawdata['[bpo_qa]_decision'] = G_UPL_rawdata['[bpo_qa]_decision'].fillna(G_UPL_rawdata['result'])
G_UPL_rawdata['Error Category 1'] = G_UPL_rawdata['error_category_1']
G_UPL_rawdata['Error Category 2'] = G_UPL_rawdata['error_category_2']
G_UPL_rawdata['Error Category 3'] = G_UPL_rawdata['error_category_3']
G_UPL_rawdata= G_UPL_rawdata[~G_UPL_rawdata['Queue Name'].isnull()]
G_UPL_rawdata = G_UPL_rawdata.sort_values(by='Moderation Date', ascending=True)
def adjust_results(row):
    if pd.isnull(row['final_result']):
        row['QA Result'] = row['2nd_result']
        row['QA Reason'] = row['2nd_rejection_reason']
        row['Mod 2 Result'] = None
        row['Mod 2 Reason'] = None
        row['Mod 1 Result'] = row['1st_result']
        row['Mod 1 Reason'] = row['1st_rejection_reason']
    else:
        row['QA Result'] = row['final_result']
        row['QA Reason'] = row['wrong_label']
        row['Mod 2 Result'] = row['result_2']
        mod_2_reason_values = [str(row[f'2_annotation.{i}']).lower() for i in range(1, 5) if not pd.isnull(row[f'2_annotation.{i}'])]
        row['Mod 2 Reason'] = ', '.join(mod_2_reason_values) if mod_2_reason_values else np.nan
        row['Mod 1 Result'] = row['result_1']
        mod_1_reason_values = [str(row[f'1_annotation.{i}']).lower() for i in range(1, 5) if not pd.isnull(row[f'1_annotation.{i}'])]
        row['Mod 2 Reason'] = ', '.join(mod_1_reason_values) if mod_1_reason_values else np.nan
    return row
G_UPL_rawdata = G_UPL_rawdata.apply(adjust_results, axis=1)
G_UPL_rawdata = G_UPL_rawdata.rename(columns={'source_file':'Week'})
def fill_final_decision(row):
    valid_values = ['Failed appeal', 'Edge case', 'Appeal successfully']
    for column in ['final_result','final_decision_(pbo)', 'bd_qa_decision', '[bpo_qa]_decision']:
        if row[column] in valid_values:
            return row[column]
        elif pd.notnull(row[column]):
            return row[column]
    return ''
G_UPL_rawdata['Final Decision'] = G_UPL_rawdata.apply(fill_final_decision, axis=1)
G_UPL_rawdata['Final Decision'] = G_UPL_rawdata['Final Decision'].fillna('Mod correct')
for k, v in aliases_final_decision.items():
    pat = '|'.join(v)
    G_UPL_rawdata.loc[G_UPL_rawdata['Final Decision'].str.contains(pat, case=False),'Final Decision'] = k
def set_policy_error(row):
    if row['Final Decision'] in ['Failed appeal', 'Edge case']:
        return row['2nd_rejection_reason']
    else:
        return row['1st_rejection_reason']
G_UPL_rawdata['Policy Error'] = G_UPL_rawdata.apply(set_policy_error, axis=1) 
def check_approval_status(value):
    for status, keywords in aliases.items():
        found_keywords = [keyword for keyword in keywords if keyword in value.lower()]
        if found_keywords:
            return status
    return None
columns_to_check = ['Mod 1 Result', 'Mod 2 Result', 'QA Result']
for column in columns_to_check:
    G_UPL_rawdata[column] = G_UPL_rawdata[column].apply(lambda x: check_approval_status(x) if isinstance(x, str) else np.nan)            
def get_moderation_error_round(row):
    if row['Final Decision'] == 'Failed appeal':
        if row['Mod 1 Result'] != row['QA Result']:
            return '1st wrong'
        elif row['Mod 2 Result'] != row['QA Result']:
            return '2nd wrong'
    return ''
G_UPL_rawdata['Moderation_Error_Round'] = G_UPL_rawdata.apply(get_moderation_error_round, axis=1)

G_UPL_lineup = G_UPL_rawdata[['Moderation Date','Queue Name','Moderators Name','Moderators Name 2','Count','Moderation_Error_Round']]
G_UPL_lineup_1 = G_UPL_lineup[['Moderation Date','Queue Name','Moderators Name','Count','Moderation_Error_Round']]
G_UPL_lineup_2 = G_UPL_lineup[['Moderation Date','Queue Name','Moderators Name 2','Count','Moderation_Error_Round']]

G_UPL_lineup_1_sample_size = G_UPL_lineup_1.groupby(['Moderation Date', 'Queue Name', 'Moderators Name'], as_index=False)['Count'].count()
G_UPL_lineup_2_sample_size = G_UPL_lineup_2.groupby(['Moderation Date', 'Queue Name', 'Moderators Name 2'], as_index=False)['Count'].count()
G_UPL_lineup_sample_size = pd.concat([G_UPL_lineup_1_sample_size, G_UPL_lineup_2_sample_size], ignore_index=True)
G_UPL_lineup_sample_size['Moderators Name'] = G_UPL_lineup_sample_size['Moderators Name'].fillna(G_UPL_lineup_sample_size['Moderators Name 2'])
G_UPL_lineup_sample_size.drop(['Moderators Name 2'], axis=1, inplace=True)

G_UPL_sample_size = pd.merge(G_UPL_lineup_sample_size,week_list,how='left',left_on='Moderation Date',right_on='Date')
G_UPL_sample_size = G_UPL_sample_size.rename(columns={'Count': 'Sample Size','UPL_Week':'Week'})
G_UPL_sample_size = G_UPL_sample_size[['Week','Moderation Date','Queue Name','Moderators Name','Sample Size']]
for week in G_UPL_sample_size['Week'].unique():
    df = G_UPL_sample_size[G_UPL_sample_size['Week'] == week]
    df.to_csv(f'Sample Size All Queues/G_UPL/{week}.csv', index=False)
G_UPL_lineup_1_mod_wrong = G_UPL_lineup_1.groupby(['Moderation Date', 'Queue Name', 'Moderators Name'])['Moderation_Error_Round'].apply(lambda x: ((x == '1st wrong')).sum()).reset_index(name='Mods Wrong')
G_UPL_lineup_2_mod_wrong = G_UPL_lineup_2[~G_UPL_lineup_2['Moderators Name 2'].isnull()]
G_UPL_lineup_2_mod_wrong = G_UPL_lineup_2_mod_wrong.groupby(['Moderation Date', 'Queue Name', 'Moderators Name 2'])['Moderation_Error_Round'].apply(lambda x: ((x == '2nd wrong')).sum()).reset_index(name='Mods Wrong')
G_UPL_lineup_mod_wrong = pd.concat([G_UPL_lineup_1_mod_wrong, G_UPL_lineup_2_mod_wrong], ignore_index=True)
G_UPL_lineup_mod_wrong['Moderators Name'] = G_UPL_lineup_mod_wrong['Moderators Name'].fillna(G_UPL_lineup_mod_wrong['Moderators Name 2'])
G_UPL_lineup_mod_wrong.drop(['Moderators Name 2'], axis=1, inplace=True)
G_UPL_mod_wrong = pd.merge(G_UPL_lineup_mod_wrong,week_list,how='left',left_on='Moderation Date',right_on='Date')
G_UPL_mod_wrong = G_UPL_mod_wrong.rename(columns={'UPL_Week':'Week'})
G_UPL_mod_wrong = G_UPL_mod_wrong[['Week','Moderation Date','Queue Name','Moderators Name','Mods Wrong']]
for week in G_UPL_mod_wrong['Week'].unique():
    df = G_UPL_mod_wrong[G_UPL_mod_wrong['Week'] == week]
    df.to_csv(f'False case All Queues/G_UPL/{week}.csv', index=False)

In [297]:
#Labeling
labeling_rawdata = pd.read_excel('Labeling/Labeling accr.xlsx')
labeling_rawdata = pd.merge(labeling_rawdata,week_list,how='left',left_on='Date',right_on='Date')
labeling_sample_size = labeling_rawdata.groupby(['Labeling_Week','Date','Category','Email'],as_index=False)['Sample size (QA output)'].apply(lambda y: y[y > 0].sum())
labeling_sample_size = labeling_sample_size[['Labeling_Week','Date','Category','Email','Sample size (QA output)']].rename(columns={'Date':'Moderation Date', 'Labeling_Week':'Week',
                                                                                                            'Category':'Queue Name',
                                                                                                            'Email':'Moderators Name',
                                                                                                            'Sample size (QA output)':'Sample Size'}) 
labeling_sample_size['Moderation Date'] = pd.to_datetime(labeling_sample_size['Moderation Date'].apply(convert_datetime))
 
labeling_mod_wrong = labeling_rawdata.groupby(['Labeling_Week','Date','Category','Email'],as_index=False)['Mod wrong tagging'].apply(lambda y: y[y > 0].sum())
labeling_mod_wrong = labeling_mod_wrong[['Labeling_Week','Date','Category','Email','Mod wrong tagging']].rename(columns={'Date':'Moderation Date', 'Labeling_Week':'Week',
                                                                                                            'Category':'Queue Name',
                                                                                                            'Email':'Moderators Name',
                                                                                                            'Mod wrong tagging':'Mods Wrong'})  
labeling_mod_wrong['Moderation Date'] = pd.to_datetime(labeling_mod_wrong['Moderation Date'].apply(convert_datetime))

In [298]:
#Side project
side_project_rawdata = merge_csv_files_sideproject('Side_project')
side_project_rawdata['mod_date'] = pd.to_datetime(side_project_rawdata['mod_date'].apply(convert_datetime))            
side_project_rawdata['Queue Name'] = side_project_rawdata['Queue Name'].replace('QA ?UG-2R-GnE?TTS-vi-VN', 'QA [UG-2R-GnE] TTS-vi-VN')
side_project_rawdata['BD QA Decision'] = side_project_rawdata['BD QA Decision'].replace('---', '')

def fill_final_decision(row):
    valid_values = ['Failed appeal', 'Edge case', 'Appeal successfully']
    
    if row['BD QA Decision'] in valid_values:
        return row['BD QA Decision']
    elif pd.notnull(row['Final Decision']):
        return row['Final Decision']
    else:
        return None
    
side_project_rawdata['Final Decision'] = side_project_rawdata.apply(fill_final_decision, axis=1)
side_project_rawdata['Final Decision'] = side_project_rawdata['Final Decision'].fillna('Mod correct')
for k, v in aliases_final_decision.items():
    pat = '|'.join(v)
    side_project_rawdata.loc[side_project_rawdata['Final Decision'].str.contains(pat, case=False),'Final Decision'] = k
Side_prj_sample_size = side_project_rawdata.groupby(by=['Week','mod_date','Queue Name','mod'],as_index=False).agg({'object id':'count'}).sort_values(by=['mod_date'],ascending=True).rename(columns={'mod_date':'Moderation Date','Queue Name':'Queue Name','mod':'Moderators Name','object id':'Sample Size'})
Side_prj_sample_size = pd.merge(Side_prj_sample_size,week_list,how='left',left_on='Moderation Date',right_on='Date')
Side_prj_sample_size = Side_prj_sample_size[['Week','Moderation Date','Queue Name','Moderators Name','Sample Size']]
for week in Side_prj_sample_size['Week'].unique():
    df = Side_prj_sample_size[Side_prj_sample_size['Week'] == week]
    df.to_csv(f'Sample Size All Queues/Side_project/{week}.csv', index=False)
#diff case - Side project
Side_prj_mod_wrong = side_project_rawdata.groupby(['mod_date','Queue Name','mod'],as_index=False)['Final Decision'].apply(lambda y: y[y =="Failed appeal"].count())
Side_prj_mod_wrong = Side_prj_mod_wrong[['mod_date','Queue Name','mod','Final Decision']].rename(columns={'mod_date':'Moderation Date',
                                                                                    'Queue Name':'Queue Name',
                                                                                    'mod':'Moderators Name',
                                                                                    'Final Decision':'Mods Wrong'})
Side_prj_mod_wrong = pd.merge(Side_prj_mod_wrong,week_list,how='left',left_on='Moderation Date',right_on='Date').rename(columns={'Mod_Week':'Week'})
Side_prj_mod_wrong = Side_prj_mod_wrong[['Week','Moderation Date','Queue Name','Moderators Name','Mods Wrong']]

In [299]:
#No-Anchor
no_anchor_rawdata = merge_csv_files_sideproject('No-Anchor')
no_anchor_rawdata['mod_date'] = pd.to_datetime(no_anchor_rawdata['mod_date'].apply(convert_datetime))            
no_anchor_rawdata['BD QA Decision'] = no_anchor_rawdata['BD QA Decision'].replace('---', '')

def fill_final_decision(row):
    valid_values = ['Failed appeal', 'Edge case', 'Appeal successfully']
    if row['BD QA Decision'] in valid_values:
        return row['BD QA Decision']
    elif pd.notnull(row['Final Decision']):
        return row['Final Decision']
    else:
        return None
    
no_anchor_rawdata['Final Decision'] = no_anchor_rawdata.apply(fill_final_decision, axis=1)
no_anchor_rawdata['Final Decision'] = no_anchor_rawdata['Final Decision'].fillna('Mod correct')
for k, v in aliases_final_decision.items():
    pat = '|'.join(v)
    no_anchor_rawdata.loc[no_anchor_rawdata['Final Decision'].str.contains(pat, case=False),'Final Decision'] = k
no_anchor_sample_size = no_anchor_rawdata.groupby(by=['Week','mod_date','Queue Name','mod'],as_index=False).agg({'object id':'count'}).sort_values(by=['mod_date'],ascending=True).rename(columns={'mod_date':'Moderation Date','Queue Name':'Queue Name','mod':'Moderators Name','object id':'Sample Size'})
no_anchor_sample_size = pd.merge(no_anchor_sample_size,week_list,how='left',left_on='Moderation Date',right_on='Date')
no_anchor_sample_size = no_anchor_sample_size[['Week','Moderation Date','Queue Name','Moderators Name','Sample Size']]
for week in no_anchor_sample_size['Week'].unique():
    df = no_anchor_sample_size[no_anchor_sample_size['Week'] == week]
    df.to_csv(f'Sample Size All Queues/No-Anchor/{week}.csv', index=False)
#diff case - no-anchor
no_anchor_mod_wrong = no_anchor_rawdata.groupby(['mod_date','Queue Name','mod'],as_index=False)['Final Decision'].apply(lambda y: y[y =="Failed appeal"].count())
no_anchor_mod_wrong = no_anchor_mod_wrong[['mod_date','Queue Name','mod','Final Decision']].rename(columns={'mod_date':'Moderation Date',
                                                                                    'Queue Name':'Queue Name',
                                                                                    'mod':'Moderators Name',
                                                                                    'Final Decision':'Mods Wrong'})
no_anchor_mod_wrong = pd.merge(no_anchor_mod_wrong,week_list,how='left',left_on='Moderation Date',right_on='Date').rename(columns={'Mod_Week':'Week'})
no_anchor_mod_wrong = no_anchor_mod_wrong[['Week','Moderation Date','Queue Name','Moderators Name','Mods Wrong']]

In [300]:
unoficial_cases_raw = input_data_csv('Random_unofficial_queues') 
unoficial_cases_raw = unoficial_cases_raw.iloc[:,:14]
unoficial_cases_raw['Mod Date'] = pd.to_datetime(unoficial_cases_raw['Mod Date'].apply(convert_datetime))   
print(unoficial_cases_raw.columns)
unoficial_sample_size = unoficial_cases_raw.groupby(by=['Week','Mod Date','Queue Name','Mod Email'],as_index=False).agg({'Object ID':'count'}).sort_values(by=['Mod Date'],ascending=True).rename(columns={'Mod Date':'Moderation Date','Queue Name':'Queue Name','Mod Email':'Moderators Name','Object ID':'Sample Size'})
unoficial_sample_size = unoficial_sample_size[['Week','Moderation Date','Queue Name','Moderators Name','Sample Size']]

unoficial_mod_wrong = unoficial_cases_raw.groupby(['Week','Mod Date','Queue Name','Mod Email'],as_index=False)['R1 Status'].apply(lambda y: y[y =='Mod Wrong'].count())
unoficial_mod_wrong = unoficial_mod_wrong[['Week','Mod Date','Queue Name','Mod Email','R1 Status']].rename(columns={'Mod Date':'Moderation Date',
                                                                                    'Queue Name':'Queue Name',
                                                                                    'Mod Email':'Moderators Name',
                                                                                    'R1 Status':'OEC/Backup Mod Wrong'})
unoficial_mod_wrong = unoficial_mod_wrong[['Week','Moderation Date','Queue Name','Moderators Name','OEC/Backup Mod Wrong']]
unoficial = pd.merge(unoficial_sample_size,unoficial_mod_wrong,how='left',left_on=['Week','Moderation Date','Queue Name','Moderators Name'],right_on=['Week','Moderation Date','Queue Name','Moderators Name'])
unoficial = unoficial.groupby(by=['Week','Moderation Date','Queue Name','Moderators Name'],as_index=False).agg({'OEC/Backup Mod Wrong':'sum'})
unoficial = unoficial[~unoficial['OEC/Backup Mod Wrong'].isnull()]

Index(['Exported Date', 'Week', 'Queue Category', 'Queue Name', 'Task ID',
       'Object ID', 'Mod Date', 'Mod Email', 'Mod Name', 'Line Manager',
       'Duration', 'R1 Result', 'Policy Error', 'R1 Status'],
      dtype='object')


In [301]:
all_sample_sizes = pd.concat([regular_sample_size, appeal_sample_size, E_UPL_sample_size, labeling_sample_size, Side_prj_sample_size,no_anchor_sample_size],axis=0)
all_sample_sizes['Moderators Name'] = all_sample_sizes['Moderators Name'].str.replace('robot_estimate@','')
all_sample_sizes = all_sample_sizes.drop_duplicates(subset=['Moderation Date','Queue Name','Moderators Name'],keep='first')
for week in all_sample_sizes['Week'].unique():
    df = all_sample_sizes[all_sample_sizes['Week'] == week]
    df.to_csv(f'Sample Size All Queues/All/{week}.csv', index=False)

all_mod_wrong =  pd.concat([regular_mod_wrong,appeal_mod_wrong,E_UPL_mod_wrong,labeling_mod_wrong,Side_prj_mod_wrong,no_anchor_mod_wrong],axis=0)
all_mod_wrong['Moderators Name'] = all_mod_wrong['Moderators Name'].str.replace('robot_estimate@','')
all_mod_wrong = all_mod_wrong.drop_duplicates()
for week in all_mod_wrong['Week'].unique():
    df = all_mod_wrong[all_mod_wrong['Week'] == week]
    df.to_csv(f'False case All Queues/All/{week}.csv', index=False)

In [302]:
#RCA - Accr Report
RCA_diff_dive_regular_appeal = regular_appeal_Rawdata.loc[(regular_appeal_Rawdata['Final Decision'] == 'Failed appeal') | (regular_appeal_Rawdata['Final Decision'] == 'Edge case'),['mod_date','Queue Name','mod','Final Decision','Error Category 1', 'Error Category 2','Error Category 3']].rename(columns={'mod_date':'Date','Queue Name':'QA QUEUE NAME','mod':'Moderator'})
RCA_diff_dive_regular_sideproject = side_project_rawdata.loc[(side_project_rawdata['Final Decision'] == 'Failed appeal') | (side_project_rawdata['Final Decision'] == 'Edge case'),['mod_date','Queue Name','mod','Final Decision','Error Category 1', 'Error Category 2','Error Category 3']].rename(columns={'mod_date':'Date','Queue Name':'QA QUEUE NAME','mod':'Moderator'})
RCA_diff_dive_e_upl = E_UPL_rawdata.loc[(E_UPL_rawdata['Final Decision']=='Failed appeal') | (E_UPL_rawdata['Final Decision']=='Edge case'),['Moderation Date','Queue Name','Moderators Name','Final Decision','Error Category 1','Error Category 2', 'Error Category 3']].rename(columns={'Moderation Date':'Date','Queue Name':'QA QUEUE NAME','Moderators Name':'Moderator'})
RCA_diff_dive_g_upl = G_UPL_rawdata.loc[(G_UPL_rawdata['Final Decision']=='Failed appeal') | (G_UPL_rawdata['Final Decision']=='Edge case'),['Moderation Date','Queue Name','Moderators Name','Final Decision','Error Category 1','Error Category 2', 'Error Category 3']].rename(columns={'1st Date':'Date','Queue Name':'QA QUEUE NAME','1st Name':'Moderator'})
RCA_diff_dive_1 = pd.concat([RCA_diff_dive_regular_appeal.reset_index(drop=True),RCA_diff_dive_regular_sideproject.reset_index(drop=True)],axis=0)
RCA_diff_dive_2 = pd.concat([RCA_diff_dive_e_upl.reset_index(drop=True),RCA_diff_dive_g_upl.reset_index(drop=True)],axis=0)
RCA_diff_dive_1 = pd.merge(RCA_diff_dive_1,week_list,left_on='Date',right_on='Date',how='left').rename(columns={'Mod_Week':'Week'})
RCA_diff_dive_2 = pd.merge(RCA_diff_dive_2,week_list,left_on='Date',right_on='Date',how='left').rename(columns={'UPL_Week':'Week'})
RCA_diff_dive_1 = RCA_diff_dive_1[['Quarter','Month','Week','QA QUEUE NAME','Moderator','Final Decision','Error Category 1', 'Error Category 2','Error Category 3']]
RCA_diff_dive_2 = RCA_diff_dive_2[['Quarter','Month','Week','QA QUEUE NAME','Moderator','Final Decision','Error Category 1', 'Error Category 2','Error Category 3']]
RCA_diff_dive =  pd.concat([RCA_diff_dive_1.reset_index(drop=True)],axis=0)
RCA_diff_dive = pd.merge(RCA_diff_dive,queue_list,how='left',left_on='QA QUEUE NAME',right_on='QA QUEUE NAME')
RCA_diff_dive_count = RCA_diff_dive.groupby(['Quarter','Month','Week', 'COMPOUND', 'MOD QUEUE NAME', 'QA QUEUE NAME', 'Moderator','Final Decision', 'Error Category 1', 'Error Category 2','Error Category 3']).agg(
    quanity_failed_appeal=('Final Decision', lambda x: (x == 'Failed appeal').sum()),
    quanity_edge_case=('Final Decision', lambda x: (x == 'Edge case').sum())).reset_index()

RCA_diff_dive_count = filter_week(RCA_diff_dive_count,filter_week_func,['Appeal','LL Product', 'CB Product','Livestream','Anchor Video','Seller', 'IPR'])
RCA_diff_dive_count

,Quarter,Month,Week,COMPOUND,MOD QUEUE NAME,QA QUEUE NAME,Moderator,Final Decision,Error Category 1,Error Category 2,Error Category 3,quanity_failed_appeal,quanity_edge_case
15400,4.0,Nov,W44 (10.29 - 11.04),Anchor Video,VN LL Anchor Video 02,[QA] VN LL Anchor Video 02,anh.pht@trans-cosmos.com.vn,Failed appeal,1. People,1.5 Policy Misinterpretation,1.5.1 Misinterpretation of the existing guideline,1,0
15401,4.0,Nov,W44 (10.29 - 11.04),Anchor Video,VN LL Anchor Video 02,[QA] VN LL Anchor Video 02,anh.tnm@trans-cosmos.com.vn,Failed appeal,1. People,1.4 Content Misinterpretation,1.4.1 Misinterpretation of the content,1,0
15402,4.0,Nov,W44 (10.29 - 11.04),Anchor Video,VN LL Anchor Video 02,[QA] VN LL Anchor Video 02,hao.th@trans-cosmos.com.vn,Failed appeal,1. People,1.3 Negligence,1.3.4 Pursuing AHT,1,0
15403,4.0,Nov,W44 (10.29 - 11.04),Anchor Video,VN LL Anchor Video 02,[QA] VN LL Anchor Video 02,huy.ltk@trans-cosmos.com.vn,Failed appeal,1. People,1.5 Policy Misinterpretation,1.5.1 Misinterpretation of the existing guideline,1,0
15404,4.0,Nov,W44 (10.29 - 11.04),Anchor Video,VN LL Anchor Video 02,[QA] VN LL Anchor Video 02,kieu.nlp@trans-cosmos.com.vn,Failed appeal,1. People,1.3 Negligence,1.3.1 Mis-click/Mis-tagging,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16223,4.0,Nov,W47 (11.19 - 11.25),Seller,VN LL Seller On Boarding,[QA]VN LL Seller On Boarding,sang.ta@trans-cosmos.com.vn,Failed appeal,1. Mod_Enforcement,1.4 Mod_Policy Misinterpretation,1.4.1 Mod misinterpretation of the existing gu...,1,0
16224,4.0,Nov,W47 (11.19 - 11.25),Seller,VN LL Seller On Boarding,[QA]VN LL Seller On Boarding,thanh.lvp@trans-cosmos.com.vn,Failed appeal,1. Mod_Enforcement,1.1 Mod_Negligence,1.1.4 Mod oversight,1,0
16225,4.0,Nov,W47 (11.19 - 11.25),Seller,VN VAT Number Check,QA VN VAT Number Check,dao.lnt@trans-cosmos.com.vn,Edge case,4. Platform/Tooling,4.2 Platform Issue,4.2.3 Access Issue,0,1
16226,4.0,Nov,W47 (11.19 - 11.25),Seller,VN VAT Number Check,QA VN VAT Number Check,dara.kd@trans-cosmos.com.vn,Edge case,4. Platform/Tooling,4.2 Platform Issue,4.2.3 Access Issue,0,2


In [303]:
#Policy Errors - Accr Report
Policy_diff_dive_regular_appeal = regular_appeal_Rawdata.loc[(regular_appeal_Rawdata['Final Decision'] == 'Failed appeal') | (regular_appeal_Rawdata['Final Decision'] == 'Edge case'), ['mod_date', 'Queue Name', 'mod','Final Decision','Policy Error']].rename(columns={'mod_date': 'Date', 'Queue Name': 'QA QUEUE NAME','mod':'Moderator'})
Policy_diff_dive_sideproject = side_project_rawdata.loc[(side_project_rawdata['Final Decision'] == 'Failed appeal') | (side_project_rawdata['Final Decision'] == 'Edge case'), ['mod_date', 'Queue Name','mod', 'Final Decision','Policy Error']].rename(columns={'mod_date': 'Date', 'Queue Name': 'QA QUEUE NAME','mod':'Moderator'})
Policy_diff_dive_e_upl = E_UPL_rawdata.loc[(E_UPL_rawdata['Final Decision']=='Failed appeal') | (E_UPL_rawdata['Final Decision']=='Edge case'),['Moderation Date','Queue Name','Moderators Name','Final Decision','Policy Error']].rename(columns={'Moderation Date':'Date','Queue Name':'QA QUEUE NAME','Moderators Name':'Moderator'})
Policy_diff_dive_g_upl = G_UPL_rawdata.loc[(G_UPL_rawdata['Final Decision']=='Failed appeal') | (G_UPL_rawdata['Final Decision']=='Edge case'),['Moderation Date','Queue Name','Moderators Name','Final Decision','Policy Error']].rename(columns={'Moderation Date':'Date','Queue Name':'QA QUEUE NAME','Moderators Name':'Moderator'})
Policy_diff_dive_1 = pd.concat([Policy_diff_dive_regular_appeal.reset_index(drop=True),Policy_diff_dive_sideproject.reset_index(drop=True)],axis=0)
Policy_diff_dive_2 = pd.concat([Policy_diff_dive_e_upl.reset_index(drop=True),Policy_diff_dive_g_upl.reset_index(drop=True)],axis=0)
Policy_diff_dive_1 = pd.merge(Policy_diff_dive_1,week_list,left_on='Date',right_on='Date',how='left').rename(columns={'Mod_Week':'Week'})
Policy_diff_dive_1 = Policy_diff_dive_1[['Quarter','Month','Week','QA QUEUE NAME','Moderator','Final Decision','Policy Error']]
Policy_diff_dive_2 = pd.merge(Policy_diff_dive_2,week_list,left_on='Date',right_on='Date',how='left').rename(columns={'UPL_Week':'Week'})
Policy_diff_dive_2 = Policy_diff_dive_2[['Quarter','Month','Week','QA QUEUE NAME','Moderator', 'Final Decision','Policy Error']]
Policy_diff_dive = pd.concat([Policy_diff_dive_1.reset_index(drop=True), Policy_diff_dive_2.reset_index(drop=True)], axis = 0)
Policy_errors = policy_errors_list.copy()
Policy_diff_dive['Policy Error'] = Policy_diff_dive['Policy Error'].str.lower()
Policy_errors_pattern = "|".join(Policy_errors)
def pattern_searcher(search_str:str, search_list:str):
    search_obj = re.search(search_list, search_str)
    if search_obj :
        return_str = search_str[search_obj.start(): search_obj.end()]
    else:
        return_str = np.nan
    return return_str
Policy_diff_dive['Policy Error'] = Policy_diff_dive['Policy Error'].astype(str).apply(lambda x: pattern_searcher(search_str=x, search_list=Policy_errors_pattern))
Policy_diff_dive = pd.merge(Policy_diff_dive,queue_list,how='left',left_on='QA QUEUE NAME',right_on='QA QUEUE NAME')
Policy_errors_count = Policy_diff_dive.groupby(['Quarter','Month','Week', 'COMPOUND', 'QA QUEUE NAME','Moderator', 'Policy Error']).agg(quanity_failed_appeal=('Final Decision', lambda x: (x == 'Failed appeal').sum()), quanity_edge_case=('Final Decision', lambda x: (x == 'Edge case').sum())).reset_index()
Policy_errors_count

,Quarter,Month,Week,COMPOUND,QA QUEUE NAME,Moderator,Policy Error,quanity_failed_appeal,quanity_edge_case
0,1.0,Feb,W5 (01.28 - 02.03),Anchor Video,[QA] VN LL Anchor Video 02,anh.tdk@trans-cosmos.com.vn,irrelevant promotion,2,0
1,1.0,Feb,W5 (01.28 - 02.03),Anchor Video,[QA] VN LL Anchor Video 02,anh.tdk@trans-cosmos.com.vn,misleading functionality and effect,1,0
2,1.0,Feb,W5 (01.28 - 02.03),Anchor Video,[QA] VN LL Anchor Video 02,chien.vm@trans-cosmos.com.vn,irrelevant promotion,1,0
3,1.0,Feb,W5 (01.28 - 02.03),Anchor Video,[QA] VN LL Anchor Video 02,chien.vm@trans-cosmos.com.vn,redirect traffic,1,0
4,1.0,Feb,W5 (01.28 - 02.03),Anchor Video,[QA] VN LL Anchor Video 02,dat.nt2@trans-cosmos.com.vn,absolute terms,1,0
...,...,...,...,...,...,...,...,...,...
13300,4.0,Oct,W43 (10.22 - 10.28),Seller,[QA]VN LL Seller On Boarding,ngan.ptt1@trans-cosmos.com.vn,possible ip infringement,0,1
13301,4.0,Oct,W43 (10.22 - 10.28),Seller,[QA]VN LL Seller On Boarding,thai.tq1@trans-cosmos.com.vn,unclear documentation,1,0
13302,4.0,Oct,W43 (10.26 - 11.01),E-UPL,[VN] Mall UP Search CB Sample QA,van.nt@trans-cosmos.com.vn,incomplete information,1,0
13303,4.0,Oct,W43 (10.26 - 11.01),E-UPL,[VN] Mall UP Search L2L Sample QA,an.lht@trans-cosmos.com.vn,unsupported products,1,0


In [304]:
## FN/ FP Rate - Accr Report - Regular/Appeal
False_errors_regular_appeal = regular_appeal_Rawdata.loc[~regular_appeal_Rawdata['Final Decision'].isnull(),['mod_date','Queue Name','mod','Final Decision','Final False Error']].rename(columns={'mod_date':'Date','Queue Name':'QA QUEUE NAME','mod':'Moderator'})
False_errors_sideproject = side_project_rawdata.loc[~side_project_rawdata['Final Decision'].isnull(),['mod_date','Queue Name','mod','Final Decision','Final False Error']].rename(columns={'mod_date':'Date','Queue Name':'QA QUEUE NAME','mod':'Moderator'})
False_errors_e_upl = E_UPL_rawdata.loc[~E_UPL_rawdata['Final Decision'].isnull(),['Moderation Date','Queue Name','Moderators Name','Final Decision','moderators_false_error']].rename(columns={'Moderation Date':'Date','Queue Name':'QA QUEUE NAME','moderators_false_error':'Final False Error','Moderators Name':'Moderator'})
False_errors_g_upl = G_UPL_rawdata.loc[~G_UPL_rawdata['Final Decision'].isnull(),['Moderation Date','Queue Name','Moderators Name','Final Decision','moderators_false_error']].rename(columns={'Moderation Date':'Date','Queue Name':'QA QUEUE NAME','moderators_false_error':'Final False Error','Moderators Name':'Moderator'})
False_errors_concat_1 = pd.concat([False_errors_regular_appeal.reset_index(drop=True), False_errors_sideproject.reset_index(drop=True)], axis = 0)
False_errors_concat_2 = pd.concat([False_errors_e_upl.reset_index(drop=True),False_errors_g_upl.reset_index(drop=True)], axis = 0)
False_errors_concat_1['Final False Error'] = False_errors_concat_1['Final False Error'].str.title()
False_errors_concat_2['Final False Error'] = False_errors_concat_2['Final False Error'].str.title()
False_errors_1 = pd.merge(False_errors_concat_1,week_list,left_on='Date',right_on='Date',how='left').rename(columns={'Mod_Week':'Week','Date_x':'Date'})
False_errors_2 = pd.merge(False_errors_concat_2,week_list,left_on='Date',right_on='Date',how='left').rename(columns={'UPL_Week':'Week'})
False_errors_1 = False_errors_1[['Quarter','Month','Week','Date','QA QUEUE NAME','Moderator','Final Decision','Final False Error']]
False_errors_2 = False_errors_2[['Quarter','Month','Week','Date','QA QUEUE NAME','Moderator','Final Decision','Final False Error']]
False_errors =  pd.concat([False_errors_1.reset_index(drop=True), False_errors_2.reset_index(drop=True)], axis = 0)
Overkill =  False_errors[(False_errors['Final Decision'].isin(['Failed appeal'])) & (False_errors['Final False Error'] == 'False Positive')]
Overkill_count = Overkill.groupby(['Quarter','Month','Week','Date','QA QUEUE NAME','Moderator']).size().reset_index(name='Overkill')
Overkill_Overall =  False_errors[(False_errors['Final Decision'].isin(['Failed appeal','Edge case'])) & (False_errors['Final False Error'] == 'False Positive')]
Overkill_Overall_count = Overkill_Overall.groupby(['Quarter','Month','Week','Date','QA QUEUE NAME','Moderator']).size().reset_index(name='Overkill_Overall')
Leakage =  False_errors[(False_errors['Final Decision'].isin(['Failed appeal'])) & (False_errors['Final False Error'] == 'False Negative')]
Leakage_count = Leakage.groupby(['Quarter','Month','Week','Date','QA QUEUE NAME','Moderator']).size().reset_index(name='Leakage')
Leakage_Overall =  False_errors[(False_errors['Final Decision'].isin(['Failed appeal','Edge case'])) & (False_errors['Final False Error'] == 'False Negative')]
Leakage_Overall_count = Leakage_Overall.groupby(['Quarter','Month','Week','Date','QA QUEUE NAME','Moderator']).size().reset_index(name='Leakage_Overall')
FP_false_errors = pd.merge(Overkill_Overall_count,Overkill_count,how='left',left_on=['Quarter','Month','Week','Date','QA QUEUE NAME','Moderator'],right_on=['Quarter','Month','Week','Date','QA QUEUE NAME','Moderator'])
FN_false_errors = pd.merge(Leakage_Overall_count,Leakage_count,how='left',left_on=['Quarter','Month','Week','Date','QA QUEUE NAME','Moderator'],right_on=['Quarter','Month','Week','Date','QA QUEUE NAME','Moderator'])
All_fn_fp = pd.merge(FP_false_errors,FN_false_errors,how='outer',on=['Quarter','Month','Week','Date','QA QUEUE NAME','Moderator'])
All_fn_fp = pd.merge(all_sample_sizes,All_fn_fp,how='left',left_on=['Week','Moderation Date','Queue Name','Moderators Name'],right_on=['Week','Date','QA QUEUE NAME','Moderator'])
All_fn_fp_final = pd.merge(All_fn_fp,queue_list,on=['QA QUEUE NAME'],how='inner')
All_fn_fp_final = All_fn_fp_final[['Quarter','Month','Week','Date','MOD QUEUE NAME','QA QUEUE NAME','Moderator','Overkill_Overall','Overkill','Leakage_Overall','Leakage','Sample Size']]
All_fn_fp_final = All_fn_fp_final.sort_values(by=['Week'], ascending = True)
All_fn_fp_final['%Overkill'] = All_fn_fp_final['Overkill'] / All_fn_fp_final['Sample Size']
All_fn_fp_final['%Leakage'] = All_fn_fp_final['Leakage'] / All_fn_fp_final['Sample Size']
All_fn_fp_final = All_fn_fp_final.groupby(by=['Quarter','Month','Week','Date','QA QUEUE NAME','Moderator'], as_index = False).agg({
   'Overkill_Overall':'sum',
   'Overkill_Overall':'sum',
   'Overkill':'sum',
   'Leakage_Overall':'sum',
   'Leakage':'sum',
   'Sample Size':'sum'})
All_fn_fp_final['%Overkill'] = All_fn_fp_final['Overkill'] / All_fn_fp_final['Sample Size']
All_fn_fp_final['%Leakage'] = All_fn_fp_final['Leakage'] / All_fn_fp_final['Sample Size']
All_fn_fp_final = pd.merge(All_fn_fp_final, queue_list,how='outer',on='QA QUEUE NAME')
All_fn_fp_final = All_fn_fp_final[['Quarter','Month','Week','Date','COMPOUND','MOD QUEUE NAME','QA QUEUE NAME','Moderator','Overkill_Overall','Overkill','Leakage_Overall','Leakage','Sample Size','%Overkill','%Leakage']]
All_fn_fp_final = All_fn_fp_final.dropna(subset='Week')
All_fn_fp_final = All_fn_fp_final.sort_values(by='Date', ascending=True)
All_fn_fp_final

,Quarter,Month,Week,Date,COMPOUND,MOD QUEUE NAME,QA QUEUE NAME,Moderator,Overkill_Overall,Overkill,Leakage_Overall,Leakage,Sample Size,%Overkill,%Leakage
2912,1.0,Jan,W1 (12.31 - 01.06),2022-12-31,LL Product,VN LL Product Initial Review,QA VN LL Product Initial Review,sang.tt1@trans-cosmos.com.vn,1.0,0.0,0.0,0.0,4.0,0.000000,0.000000
9645,1.0,Jan,W1 (12.31 - 01.06),2022-12-31,Livestream,VN LL Live Report 02,[QA] VN LL Live Report 02,nhu.nvq@trans-cosmos.com.vn,1.0,1.0,0.0,0.0,6.0,0.166667,0.000000
9646,1.0,Jan,W1 (12.31 - 01.06),2022-12-31,Livestream,VN LL Live Report 02,[QA] VN LL Live Report 02,quyen.tt4@trans-cosmos.com.vn,0.0,0.0,1.0,1.0,2.0,0.000000,0.500000
8855,1.0,Jan,W1 (12.31 - 01.06),2022-12-31,Livestream,VN LL Live 02,[QA] VN LL Live 02,an.hb2@trans-cosmos.com.vn,1.0,1.0,0.0,0.0,2.0,0.500000,0.000000
8856,1.0,Jan,W1 (12.31 - 01.06),2022-12-31,Livestream,VN LL Live 02,[QA] VN LL Live 02,gia.pn@trans-cosmos.com.vn,0.0,0.0,1.0,1.0,3.0,0.000000,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15889,4.0,Dec,W47 (11.23 - 11.29),2023-11-28,E-UPL,[VN] Mall UP Search L2L Sample,[VN] Mall UP Search L2L Sample QA,anh.tnm@trans-cosmos.com.vn,0.0,0.0,1.0,1.0,9.0,0.000000,0.111111
4849,4.0,Dec,W48 (11.26 - 12.02),2023-11-28,LL Product,VN LL Product Initial Review,QA VN LL Product Initial Review,huyen.nt5@trans-cosmos.com.vn,0.0,0.0,1.0,1.0,2.0,0.000000,0.500000
15980,4.0,Dec,W48 (11.26 - 12.02),2023-11-28,IPR,VN Product IPR initial review,QA VN Product IPR initial review,thien.ld@trans-cosmos.com.vn,0.0,0.0,1.0,1.0,6.0,0.000000,0.166667
13065,4.0,Dec,W48 (11.26 - 12.02),2023-11-28,CB Product,VN CB Product Buffer,[QA]VN CB Product Buffer,duong.pt@trans-cosmos.com.vn,0.0,0.0,1.0,1.0,3.0,0.000000,0.333333


In [305]:
compound_filter = ['LL Product','CB Product','Seller','IPR','Appeal','Livestream','Anchor Video']
mod_accr_overall = pd.merge(all_sample_sizes,all_mod_wrong,how='left',
                            left_on=['Week', 'Moderation Date', 'Queue Name', 'Moderators Name'],
                            right_on=['Week', 'Moderation Date', 'Queue Name', 'Moderators Name'])
mod_accr_overall['Mods Wrong'] = mod_accr_overall['Mods Wrong'].fillna(0)       
mod_accr_overall_copy = pd.merge(mod_accr_overall,week_list,how='left',left_on=['Week','Moderation Date'],right_on=['Mod_Week','Date'])
mod_accr_overall_copy = pd.merge(mod_accr_overall_copy,queue_list,how='left',left_on='Queue Name',right_on='QA QUEUE NAME')
mod_accr_overall_copy = mod_accr_overall_copy[['Quarter','Month','Week', 'Moderation Date','COMPOUND','MOD QUEUE NAME','QA QUEUE NAME','Moderators Name','Sample Size','Mods Wrong']]
mod_accr_overall_copy = mod_accr_overall_copy[~mod_accr_overall_copy['Month'].isnull()]
mod_accr_overall_copy = mod_accr_overall_copy.drop_duplicates()
mod_accr_overall_final_filter = filter_week(mod_accr_overall_copy,filter_week_func,['Appeal','LL Product', 'CB Product','Livestream','Anchor Video','Seller', 'IPR'])
mod_accr_overall_final_filter

,Quarter,Month,Week,Moderation Date,COMPOUND,MOD QUEUE NAME,QA QUEUE NAME,Moderators Name,Sample Size,Mods Wrong
244888,4.0,Nov,W44 (10.29 - 11.04),2023-10-29,CB Product,VN CB Shop Decoration Risk Review,QA VN CB Shop Decoration Risk Review,phung.nhk@trans-cosmos.com.vn,1.0,0.0
244889,4.0,Nov,W44 (10.29 - 11.04),2023-10-29,IPR,VN LL Anchor Video Counterfeit Key Frame,QA VN LL Anchor Video Counterfeit Key Frame,an.hb2@trans-cosmos.com.vn,1.0,0.0
244890,4.0,Nov,W44 (10.29 - 11.04),2023-10-29,IPR,VN LL Anchor Video Counterfeit Key Frame,QA VN LL Anchor Video Counterfeit Key Frame,bang.pt@trans-cosmos.com.vn,2.0,0.0
244891,4.0,Nov,W44 (10.29 - 11.04),2023-10-29,IPR,VN LL Anchor Video Counterfeit Key Frame,QA VN LL Anchor Video Counterfeit Key Frame,bich.ntn2@trans-cosmos.com.vn,5.0,0.0
244892,4.0,Nov,W44 (10.29 - 11.04),2023-10-29,IPR,VN LL Anchor Video Counterfeit Key Frame,QA VN LL Anchor Video Counterfeit Key Frame,chau.plb@trans-cosmos.com.vn,5.0,0.0
...,...,...,...,...,...,...,...,...,...,...
338895,4.0,Nov,W47 (11.19 - 11.25),2023-11-25,Anchor Video,VN LL No-Anchor Video,QA VN LL No-Anchor Video,dong.ltk@trans-cosmos.com.vn,1.0,0.0
338896,4.0,Nov,W47 (11.19 - 11.25),2023-11-25,Anchor Video,VN LL No-Anchor Video,QA VN LL No-Anchor Video,giang.lt3@trans-cosmos.com.vn,1.0,0.0
338897,4.0,Nov,W47 (11.19 - 11.25),2023-11-25,Anchor Video,VN LL No-Anchor Video,QA VN LL No-Anchor Video,hau.nq@trans-cosmos.com.vn,2.0,0.0
338898,4.0,Nov,W47 (11.19 - 11.25),2023-11-25,Livestream,VN LL Live No-Anchor Commercial Content,QA VN LL Live No-Anchor Commercial Content,tu.lq@trans-cosmos.com.vn,2.0,0.0


In [306]:
data_full_alternation.columns

Index(['EffectDate', 'Email', 'LineManager', 'Batch', 'Task',
       'ProductionTenure'],
      dtype='object')

In [307]:
# mod_sample = pd.read_excel('mod no moderation.xlsx')
# mod_sample['Date'] = pd.to_datetime(mod_sample['Date'].apply(convert_datetime))
# print(mod_sample.columns)
# mod_sample_ratio = pd.merge(mod_sample,mod_accr_overall_final_filter,how='left',right_on=['Moderation Date','MOD QUEUE NAME','Moderators Name'],left_on=['Date','Queue Title','Moderator'])
# mod_sample_ratio = mod_sample_ratio[['Week','Date','COMPOUND','Queue Title','Moderator','No. of Moderation Tasks','Sample Size','Mods Wrong']]
# mod_sample_ratio.to_excel('mod samples ratio.xlsx',index=False)

In [308]:
select_columns = mod_accr_overall.columns
unoficial_filter_time = filter_time(unoficial,'Moderation Date',from_DATE,to_DATE,unoficial.columns)
mod_accr_overall_filter_time = filter_time(mod_accr_overall,'Moderation Date',from_DATE,to_DATE,mod_accr_overall.columns)
try:
   mod_accr_overall_filter_time.to_excel(f'Mod KPI {from_DATE}_{to_DATE}.xlsx',index=False)
   unoficial.to_excel(f'Unofficial false cases {from_DATE}_{to_DATE}.xlsx',index=False)
except:
   pass
mod_accr_overall_filter_time_copy = filter_time(mod_accr_overall_copy,'Moderation Date',from_DATE,to_DATE,mod_accr_overall_copy.columns)
mod_accr_overall_filter_time_copy.columns
mod_accr_overall_filter_time_copy

,Quarter,Month,Week,Moderation Date,COMPOUND,MOD QUEUE NAME,QA QUEUE NAME,Moderators Name,Sample Size,Mods Wrong
253105,4.0,Nov,W45 (11.05 - 11.11),2023-11-05,IPR,VN CB Trademark New,QA VN CB Trademark New,bao.dtq@trans-cosmos.com.vn,1.0,0.0
253106,4.0,Nov,W45 (11.05 - 11.11),2023-11-05,IPR,VN LL Anchor Video Counterfeit Key Frame,QA VN LL Anchor Video Counterfeit Key Frame,anh.tdk@trans-cosmos.com.vn,2.0,0.0
253107,4.0,Nov,W45 (11.05 - 11.11),2023-11-05,IPR,VN LL Anchor Video Counterfeit Key Frame,QA VN LL Anchor Video Counterfeit Key Frame,bich.ntn2@trans-cosmos.com.vn,5.0,0.0
253108,4.0,Nov,W45 (11.05 - 11.11),2023-11-05,IPR,VN LL Anchor Video Counterfeit Key Frame,QA VN LL Anchor Video Counterfeit Key Frame,danh.vt1@trans-cosmos.com.vn,1.0,0.0
253109,4.0,Nov,W45 (11.05 - 11.11),2023-11-05,IPR,VN LL Anchor Video Counterfeit Key Frame,QA VN LL Anchor Video Counterfeit Key Frame,dung.ltm@trans-cosmos.com.vn,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...
338895,4.0,Nov,W47 (11.19 - 11.25),2023-11-25,Anchor Video,VN LL No-Anchor Video,QA VN LL No-Anchor Video,dong.ltk@trans-cosmos.com.vn,1.0,0.0
338896,4.0,Nov,W47 (11.19 - 11.25),2023-11-25,Anchor Video,VN LL No-Anchor Video,QA VN LL No-Anchor Video,giang.lt3@trans-cosmos.com.vn,1.0,0.0
338897,4.0,Nov,W47 (11.19 - 11.25),2023-11-25,Anchor Video,VN LL No-Anchor Video,QA VN LL No-Anchor Video,hau.nq@trans-cosmos.com.vn,2.0,0.0
338898,4.0,Nov,W47 (11.19 - 11.25),2023-11-25,Livestream,VN LL Live No-Anchor Commercial Content,QA VN LL Live No-Anchor Commercial Content,tu.lq@trans-cosmos.com.vn,2.0,0.0


In [309]:
ipr_transfer_tracker

,EffectDate,Email,LineManager,Task,IPR_transfer,IPR_Seniority
3797,2022-09-05,ai.btm@trans-cosmos.com.vn,NGUYỄN NGỌC TRÍ,0,No,NaT
3798,2022-09-06,ai.btm@trans-cosmos.com.vn,NGUYỄN NGỌC TRÍ,0,No,NaT
3799,2022-09-07,ai.btm@trans-cosmos.com.vn,NGUYỄN NGỌC TRÍ,0,No,NaT
3800,2022-09-08,ai.btm@trans-cosmos.com.vn,NGUYỄN NGỌC TRÍ,0,No,NaT
3801,2022-09-09,ai.btm@trans-cosmos.com.vn,NGUYỄN NGỌC TRÍ,0,No,NaT
...,...,...,...,...,...,...
209833,2023-05-18,yen.tp@trans-cosmos.com.vn,NGUYỄN ĐỖ PHÚ YÊN,Live_ds,No,NaT
209834,2023-05-19,yen.tp@trans-cosmos.com.vn,NGUYỄN ĐỖ PHÚ YÊN,Live_ds,No,NaT
209835,2023-05-20,yen.tp@trans-cosmos.com.vn,NGUYỄN ĐỖ PHÚ YÊN,Live_ds,No,NaT
209836,2023-05-21,yen.tp@trans-cosmos.com.vn,NGUYỄN ĐỖ PHÚ YÊN,Live_ds,No,NaT


In [310]:
#NEWBIE/OLDBIE
samples_ppe = input_data_csv("Sample Size All Queues/All")
false_ppe = input_data_csv("False case All Queues/All")

samples_ppe['Moderation Date'] = pd.to_datetime(samples_ppe['Moderation Date'].apply(convert_datetime))
false_ppe['Moderation Date'] = pd.to_datetime(false_ppe['Moderation Date'].apply(convert_datetime))

compound_filter = ['Anchor Video','Livestream','CB Product','LL Product','Seller','IPR','Appeal']
mod_accr_overall = pd.merge(samples_ppe,false_ppe,how='left',
                            left_on=['Week', 'Moderation Date', 'Queue Name', 'Moderators Name'],
                            right_on=['Week', 'Moderation Date', 'Queue Name', 'Moderators Name'])
mod_accr_overall['Mods Wrong'] = mod_accr_overall['Mods Wrong'].fillna(0)              
mod_accr_overall = pd.merge(mod_accr_overall,queue_list,how='left',left_on='Queue Name',right_on='QA QUEUE NAME')
mod_accr_overall = mod_accr_overall[['Week', 'Moderation Date','CATEGORY','COMPOUND', 'QA QUEUE NAME', 'Moderators Name','Sample Size','Mods Wrong']]
mod_accr_overall = mod_accr_overall[mod_accr_overall['COMPOUND'].isin(compound_filter)]
mod_accr_overall = mod_accr_overall.sort_values(by='Moderation Date',ascending=True)
newbie_oldbie_tracker_non_ipr_temp = mod_accr_overall[mod_accr_overall['COMPOUND']!='IPR']
newbie_oldbie_tracker_ipr_temp = mod_accr_overall[mod_accr_overall['COMPOUND']=='IPR']

ipr_transfer_tracker = ipr_transfer_tracker[ipr_transfer_tracker['IPR_transfer']=='Yes']

newbie_oldbie_tracker_non_ipr = pd.merge(newbie_oldbie_tracker_non_ipr_temp,data_full_alternation,how='left',left_on=['Moderation Date','Moderators Name'],right_on=['EffectDate','Email'])
newbie_oldbie_tracker_non_ipr.loc[newbie_oldbie_tracker_non_ipr['ProductionTenure'] >= 90, 'Seniority'] = 'oldbie'
newbie_oldbie_tracker_non_ipr.loc[newbie_oldbie_tracker_non_ipr['ProductionTenure'] < 90, 'Seniority'] = 'newbie'
newbie_oldbie_tracker_non_ipr.drop(columns=['EffectDate', 'Email', 'LineManager', 'Task','Batch','ProductionTenure'], inplace=True)

newbie_oldbie_tracker_ipr = pd.merge(newbie_oldbie_tracker_ipr_temp,ipr_transfer_tracker,how='left',left_on=['Moderators Name'],right_on=['Email'])
newbie_oldbie_tracker_ipr_initially_temp = newbie_oldbie_tracker_ipr[newbie_oldbie_tracker_ipr['IPR_transfer'].isnull()]
newbie_oldbie_tracker_ipr_initially_temp.drop(columns=['EffectDate', 'Email', 'LineManager', 'Task', 'IPR_transfer', 'IPR_Seniority'], inplace=True)
newbie_oldbie_tracker_ipr_initially = pd.merge(newbie_oldbie_tracker_ipr_initially_temp,data_full_alternation,how='left',left_on=['Moderation Date','Moderators Name'],right_on=['EffectDate','Email'])
newbie_oldbie_tracker_ipr_initially.loc[newbie_oldbie_tracker_ipr_initially['ProductionTenure'] >= 90, 'Seniority'] = 'oldbie'
newbie_oldbie_tracker_ipr_initially.loc[newbie_oldbie_tracker_ipr_initially['ProductionTenure'] < 90, 'Seniority'] = 'newbie'
newbie_oldbie_tracker_ipr_initially.drop(columns=['EffectDate', 'Email', 'LineManager', 'Task','Batch','ProductionTenure'], inplace=True)

newbie_oldbie_tracker_ipr_transfer = newbie_oldbie_tracker_ipr[~newbie_oldbie_tracker_ipr['IPR_transfer'].isnull()]
newbie_oldbie_tracker_ipr_transfer = newbie_oldbie_tracker_ipr_transfer[newbie_oldbie_tracker_ipr_transfer['Moderation Date'] >= newbie_oldbie_tracker_ipr_transfer['EffectDate']]
newbie_oldbie_tracker_ipr_transfer = newbie_oldbie_tracker_ipr_transfer.drop_duplicates()
conditions = [
    (newbie_oldbie_tracker_ipr_transfer['Moderation Date'] >= newbie_oldbie_tracker_ipr_transfer['EffectDate']) &
    (newbie_oldbie_tracker_ipr_transfer['Moderation Date'] <= newbie_oldbie_tracker_ipr_transfer['IPR_Seniority']),
    (newbie_oldbie_tracker_ipr_transfer['Moderation Date'] > newbie_oldbie_tracker_ipr_transfer['IPR_Seniority'])]
values = ['newbie', 'oldbie']
newbie_oldbie_tracker_ipr_transfer['Seniority'] = np.select(conditions, values, default='other')
newbie_oldbie_tracker_ipr_transfer.drop(columns=['EffectDate', 'Email', 'LineManager', 'Task','IPR_transfer','IPR_Seniority'], inplace=True)

newbie_oldbie_tracker = pd.concat([newbie_oldbie_tracker_non_ipr,newbie_oldbie_tracker_ipr_initially,newbie_oldbie_tracker_ipr_transfer],axis=0)
newbie_oldbie_tracker_download = filter_week(newbie_oldbie_tracker,filter_week_func,['Appeal','LL Product', 'CB Product','Livestream','Anchor Video','Seller', 'IPR'])
newbie_oldbie_tracker_download.to_excel('newbie_oldbie_tracker_download.xlsx', index=False )

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_7908\3142622352.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newbie_oldbie_tracker_ipr_initially_temp.drop(columns=['EffectDate', 'Email', 'LineManager', 'Task', 'IPR_transfer', 'IPR_Seniority'], inplace=True)


In [311]:
exclude_acc = pd.read_csv('nov_ag_list_without_manager.csv')
exclude_acc = exclude_acc[exclude_acc['filter_col'] == 1]
exclude_acc['max_date'] = pd.to_datetime(exclude_acc['max_date'].apply(convert_datetime))
exclude_acc['min_date'] = pd.to_datetime(exclude_acc['min_date'].apply(convert_datetime))
exclude_acc = exclude_acc[['Email', 'min_date', 'max_date']]

kpi_result = pd.DataFrame()

for index, row in exclude_acc.iterrows():
    temp_df = mod_accr_overall_filter_time[
        (mod_accr_overall_filter_time['Moderation Date'] >= row['min_date']) &
        (mod_accr_overall_filter_time['Moderation Date'] <= row['max_date']) &
        (mod_accr_overall_filter_time['Moderators Name'] == row['Email'])
    ]
    if not temp_df.empty:
        kpi_result = pd.concat([kpi_result, temp_df])

kpi_result = kpi_result.drop_duplicates()
kpi_result.to_excel(f'Mod KPI {from_DATE}_{to_DATE}.xlsx',index=False)

In [312]:
# mod_sample_analyst = pd.merge(live_prod,mod_accr_overall_filter_time_copy,how='left',left_on=['Date','QA QUEUE NAME', 'Moderator'],right_on=['Moderation Date', 'QA QUEUE NAME','Moderators Name'])
# mod_sample_analyst = mod_sample_analyst[['Mod_Week', 'Date', 'QA QUEUE NAME','Moderator','No. of Moderation Tasks', 'Sample Size', 'Mods Wrong']]
# mod_sample_analyst.to_excel('mod_live_sample_analyst.xlsx',index=False)
# mod_sample_analyst

In [313]:
# mod_accr_overall_final_filter = filter_week(mod_accr_overall_copy,filter_week_func,compound_filter)
# All_fn_fp_final_filter = filter_week(All_fn_fp_final,filter_week_func,compound_filter)

In [314]:
all_sheets_mod = {'Acc': mod_accr_overall_final_filter,'RCA':RCA_diff_dive_count}

mod_writer = pd.ExcelWriter('mod.xlsx', engine='xlsxwriter')

for sheet_name in all_sheets_mod.keys():
    all_sheets_mod[sheet_name].to_excel(mod_writer, sheet_name=sheet_name, index=False)

mod_writer.close()

In [315]:
# overall_KPI_Internal = mod_accr_overall_copy.groupby(by=['CATEGORY'],as_index=False).agg({'Sample Size':'sum','Mods Wrong':'sum'})
# overall_KPI_Internal

In [316]:
data_full_alternation.columns

Index(['EffectDate', 'Email', 'LineManager', 'Batch', 'Task',
       'ProductionTenure'],
      dtype='object')

In [317]:
mod_accr_overall_linemanager = mod_accr_overall_copy.groupby(by=['Moderation Date','QA QUEUE NAME','Moderators Name'], as_index=False).agg({'Sample Size':'sum','Mods Wrong':'sum'})
mod_accr_overall_linemanager = filter_time(mod_accr_overall_linemanager,'Moderation Date',from_DATE,to_DATE,['Moderation Date','QA QUEUE NAME','Moderators Name','Sample Size','Mods Wrong'])
mod_accr_overall_linemanager = mod_accr_overall_linemanager[['Moderation Date','QA QUEUE NAME','Moderators Name','Sample Size','Mods Wrong']]
mod_accr_overall_linemanager = pd.merge(mod_accr_overall_linemanager,data_full_alternation,how='left',left_on=['Moderation Date','Moderators Name'],right_on=['EffectDate','Email'])
mod_accr_overall_linemanager = mod_accr_overall_linemanager[~mod_accr_overall_linemanager['Role'].isnull()]
mod_accr_overall_linemanager = pd.merge(mod_accr_overall_linemanager,week_list,how='left',left_on='Moderation Date',right_on='Date')

KeyError: 'Role'

In [ ]:
mod_accr_overall_linemanager = mod_accr_overall_linemanager.dropna(subset=['LineManager', 'Report_Week'])
mod_accr_overall_linemanager = mod_accr_overall_linemanager[['LineManager','Role','Batch','ProductionTenure','Report_Week','Moderation Date','QA QUEUE NAME','Moderators Name','Sample Size','Mods Wrong']]

In [ ]:
filter_week_func = {'W44'}
def filter_week(df,W):
    df = df[df['Report_Week'].str.contains('|'.join(W))]
    return df
mod_accr_overall_linemanager_2 = filter_week(mod_accr_overall_linemanager,filter_week_func)
mod_accr_overall_linemanager_2 = mod_accr_overall_linemanager_2.groupby(by=['LineManager','Report_Week','Moderation Date','Moderators Name'],as_index=False).agg({'Sample Size':'sum','Mods Wrong':'sum'})

In [ ]:
mod_accr_overall_linemanager_2.to_excel(f'mod accr line manager {filter_week_func}.xlsx',index=False)